In [1]:
import ast
#input matched pairs
import numpy as np
import pandas as pd
from pymatgen.io.vasp import outputs
from pymatgen.io.vasp.outputs import Outcar, Vasprun
from pymatgen.electronic_structure.dos import CompleteDos
from pymatgen.electronic_structure.bandstructure import BandStructure, BandStructureSymmLine, Kpoint
import BandCrossing
import eFermiMat
%matplotlib inline
#dos from bands
#define histogram function
FINGERPRINT_LENGTH = 200
def calc_all_k_branches_histogram_fingerprint(bands_dict, fingerprint_length=FINGERPRINT_LENGTH, e_range=8., normalize=False, shifted=True):

    fingerprint_p = np.zeros(fingerprint_length)
    fingerprint_m = np.zeros(fingerprint_length)
    
    k_points = bands_dict["efermi"]
    efermi =  bands_dict["efermi"]
    energies_p = np.array(bands_dict["bands"]["1"])
    energies_m = np.array(bands_dict["bands"]["-1"])

    if shifted:
        energies_p -= bands_dict["efermi"]
        energies_m -= bands_dict["efermi"]

    counts_p, edges_p = np.histogram(energies_p, bins=fingerprint_length, range=[efermi - e_range, efermi + e_range], density=normalize)
    counts_m, edges_m = np.histogram(energies_m, bins=fingerprint_length, range=[efermi - e_range, efermi + e_range], density=normalize)

    fingerprint_p = counts_p
    fingerprint_m = counts_m
    
    return fingerprint_p, fingerprint_m



/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [ ]:
path_pairs = '../data/matches_output/matches_output_jarvis.xlsx'
df_pairs = pd.read_excel(path_pairs)
root_dir = '../data/monolayer_poscar_jarvis/'
overlap_ind = []
for i in range(len(df_pairs)):
    layers = [df_pairs.iloc[i]['original_id1'].split('-')[1],df_pairs.iloc[i]['original_id2'].split('-')[1]]
    names = [ast.literal_eval(df_pairs.iloc[i]['formula'])[0],ast.literal_eval(df_pairs.iloc[i]['formula'])[1]]
    folder1 = root_dir + 'POSCAR_' + names[0] + '_' + layers[0]+'/'
    folder2 = root_dir + 'POSCAR_' + names[1] + '_' + layers[1]+'/'
    kpth = folder1 + 'bands/KPOINTS'
    path_mono1 = folder1 + 'bands_run/vasprun.xml'
    path_mono2 = folder2 + 'bands_run/vasprun.xml'
    path_mono1_dos = folder1 + 'dos_run/vasprun.xml'
    path_mono2_dos = folder2 + 'dos_run/vasprun.xml'
    try:
        vrb1 = Vasprun(path_mono1,
                                 parse_dos = False,parse_eigen = True, parse_projected_eigen = True, parse_potcar_file= False,
                                 occu_tol= 1e-08, separate_spins = True, exception_on_bad_xml= True)
        vrb2 = Vasprun(path_mono2,
                                 parse_dos = False,parse_eigen = True, parse_projected_eigen = True, parse_potcar_file= False,
                                 occu_tol= 1e-08, separate_spins = True, exception_on_bad_xml= True)
        vrd1 = Vasprun(path_mono1_dos,
                                 parse_dos = True)
        vrd2 = Vasprun(path_mono2_dos,
                                 parse_dos = True)
        #locpot = outputs.Locpot.from_file(path_mono1_dos+'/LOCPOT')
        #Evac1 = (max(np.mean(np.mean(locpot.data['total'],axis=0),axis=0)))
        dos1 = vrd1.tdos.as_dict()
        dos2 = vrd2.tdos.as_dict()
        ef1 = dos1['efermi']
        ef2 = dos2['efermi']
        band1 = vrb1.get_band_structure(kpoints_filename = kpth, efermi=vrd1.efermi, line_mode = True).as_dict()
        band2 = vrb2.get_band_structure(kpoints_filename = kpth, efermi=vrd2.efermi, line_mode = True).as_dict()
        Evac1 = 0
        Evac2 = 0
        DelEv = Evac1 - Evac2
        np_mat2_eigs_up = np.array(band2['bands']['1']) + DelEv
        np_mat2_eigs_dn = np.array(band2['bands']['-1']) + DelEv
        np_mat2_dosE = np.array(dos2['energies']) + DelEv
        mat2_eigs_up = np_mat2_eigs_up.tolist()
        mat2_eigs_dn = np_mat2_eigs_dn.tolist()
        mat2_dosE = np_mat2_dosE.tolist()
        shifted_eig_mat2 = {'1':mat2_eigs_up, '-1':mat2_eigs_dn}
        shifted_dos_mat2 = {'energies':mat2_dosE, 'densities': dos2['densities']}
        kpoints = band1['kpoints']
        #calculate Fermi level
        new_fermi = eFermiMat.HeteroFermiCalc(mat1_dos = dos1 ,ef1 = ef1, mat2_dos = shifted_dos_mat2 , ef2 = ef2+DelEv , dos_step = 100)
        #print(new_fermi)
        _, overlap_index_temp = BandCrossing.overlap_ind_hsl(new_fermi,DelEv,band1['bands'],band2['bands'],kpoints,en_range_width = 6)
        overlap_ind.append(overlap_index_temp)
    except Exception as e:
        print(e)

df_pairs['overlap_ind'] = overlap_ind

In [ ]:
mat_new = df_pairs.sort_values(by='elastic_energy')

# Define the file path and name
file_path = '/Users/user/Documents/pycode/Lattice matching/jarvis_overlap_6eV.xlsx'

# Output the DataFrame to an Excel file
mat_new.to_excel(file_path, index=False)

print(f"DataFrame has been written to {file_path}")